In [11]:
import torch
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Label
import numpy as np
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
print(f"Using Torch version {torch.__version__}.  CUDA is {'available' if torch.cuda.is_available() else 'not available'}.")
print(f"Using bokeh version {bokeh.__version__}.")
import pandas as pd
output_notebook()

Using Torch version 1.13.1.post200.  CUDA is available.
Using bokeh version 3.1.0.


Loading BokehJS ...

In [12]:
class LogR(torch.nn.Module):
    """A simple 15x1 logistic regression model."""
    def __init__(self):
        super().__init__()
        self.logistic=torch.nn.Linear(15,1,dtype=torch.float64)

    def forward(self,x):
        return torch.sigmoid(self.logistic(x))

In [43]:
df = pd.read_csv('ifood_df.csv',delimiter=',')
features = df.columns[[0,4,5,6,7,8,9,10,11,12,13,14,24,36,37]]
df[features] = (df[features]-df[features].mean())/df[features].std()
x_train, x_test, y_train, y_test = train_test_split(df[features].values, df['Response'].values)
device='cpu'


In [44]:
model = LogR().to(device)
print(model)
for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

LogR(
  (logistic): Linear(in_features=15, out_features=1, bias=True)
)
Layer: logistic.weight | Size: torch.Size([1, 15]) | Values : tensor([[-0.0925,  0.0294,  0.0025,  0.1708,  0.1358, -0.0036, -0.0292,  0.1510,
          0.0529,  0.2398,  0.1922,  0.2253,  0.1348,  0.0635, -0.0300]],
       dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: logistic.bias | Size: torch.Size([1]) | Values : tensor([-0.0728], dtype=torch.float64, grad_fn=<SliceBackward0>) 



In [45]:
Xt = torch.tensor(x_train,dtype=torch.float64,device=device)
Yt = torch.tensor(y_train,dtype=torch.float64,device=device)
criterion = torch.nn.functional.binary_cross_entropy
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)


In [46]:
def train(model, criterion, optimizer, Xt, Yt):
    """One step through the training loop"""
    # reset the gradient calculations
    optimizer.zero_grad()

    # forward pass
    predicted = model(Xt)
    
    # compute the loss
    loss = criterion(torch.squeeze(predicted),Yt)
    
 

    # compute the gradients by backward propogation
    loss.backward()        
        
    # adjust the weights
    optimizer.step()
    
    return loss.item()

In [47]:
def plot_loss(model, data, target, learning_rate=.0001, threshold=1e-6, max_iter=100000):
    """Run the model and collect the losses; return a figure"""
    
    criterion = torch.nn.functional.binary_cross_entropy
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    losses = []
    prior_loss=1000000
    for i in tqdm.tqdm(range(max_iter)):
        loss = train(model,criterion,optimizer,Xt,Yt)
        losses.append(loss)
        if abs(loss-prior_loss) < threshold:
            break
        prior_loss = loss

 
    f=figure(title=f"Loss over time: lr={learning_rate} threshold={threshold}",x_axis_label="Epoch",y_axis_label="Loss")
    f.line(x=list(range(len(losses))),y=losses)
    with torch.no_grad():
        prediction = model(Xt).round()
        print(f"Accuracy is {((prediction - Yt.reshape(-1,1))==0).sum()/Xt.shape[0]}")
    
    return f

In [48]:
model = LogR().to(device)
show(plot_loss(model, Xt, Yt))

 69%|██████▉   | 69093/100000 [00:18<00:08, 3830.60it/s]


Accuracy is 0.8614640235900879
